# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[28037, 28038, 28039, 28040, 28041, 28042, 28043, 28044, 28045, 28046]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
28037 Will any major Atlantic hurricanes (Category 3 or higher) occur in September 2024?
Weather prediction
28038 Will exactly 1 major Atlantic hurricane (Category 3 or higher) occur in September 2024?
Weather prediction
28039 Will exactly 2 major Atlantic hurricanes (Category 3 or higher) occur in September 2024?
Weather prediction
28040 Will exactly 3 major Atlantic hurricane (Category 3 or higher) occur in September 2024?
Weather prediction
28041 Will 4 or more major Atlantic hurricane (Category 3 or higher) occur in September 2024?
Weather prediction
28042 Will AfD's vote share be more than 32% in the 2024 Brandenburg state election?
Election outcome
28043 Will AfD's vote share be more than 30% and less than or equal to 32% in the 2024 Brandenburg state election?
Election outcome
28044 Will AfD's vote share be more than 28% and less than or equal to 30% in the 2024 Brandenburg state election?
Election outcome
28045 Will AfD's vote share be less than or equal to 28% in 

In [9]:
for ifp in ifps.values():
    try:
        print(ifp.id, ifp.forecast)
    except:
        print(ifp.id, 'none')

28037 40
28038 30
28039 20
28040 10
28041 5
28042 20
28043 25
28044 40
28045 65
28046 45


In [10]:
fcst.report()

28037 Will any major Atlantic hurricanes (Category 3 or higher) occur in September 2024?
Forecast 40
Rationale Given the historically active period in mid-September and upcoming potentially favorable conditions, the chance of at least one major hurricane forming is still moderately high despite the lull. 

28038 Will exactly 1 major Atlantic hurricane (Category 3 or higher) occur in September 2024?
Forecast 30
Rationale The exact occurrence of one major hurricane is less likely due to either more or less than one major hurricane forming. The chance remains moderate due to historical hurricane peak period and potential favorable conditions. 

28039 Will exactly 2 major Atlantic hurricanes (Category 3 or higher) occur in September 2024?
Forecast 20
Rationale The likelihood of exactly two major hurricanes forming is relatively lower, considering the current lull and the time remaining in September. 

28040 Will exactly 3 major Atlantic hurricane (Category 3 or higher) occur in September 2

In [11]:
fcst.upload()

Prediction posted for  28037
Comment posted for  28037
Prediction posted for  28038
Comment posted for  28038
Prediction posted for  28039
Comment posted for  28039
Prediction posted for  28040
Comment posted for  28040
Prediction posted for  28041
Comment posted for  28041
Prediction posted for  28042
Comment posted for  28042
Prediction posted for  28043
Comment posted for  28043
Prediction posted for  28044
Comment posted for  28044
Prediction posted for  28045
Comment posted for  28045
Prediction posted for  28046
Comment posted for  28046
